In [1]:
#using Pkg

#Pkg.add("Plots")
#Pkg.add("SparseArrays")
#Pkg.add("IterativeSolvers")
#Pkg.add("BenchmarkTools")
#Pkg.add("DelimitedFiles")
using Plots, SparseArrays, IterativeSolvers, BenchmarkTools,DelimitedFiles,DelimitedFiles,Tables


include("functions.jl");

## Solver for the Poisson equation

In [2]:
function benchmark_poisson(dims,L,tol,direct)    
    norms = []
    times = []
    us = []
    for dim in dims
        u_a,Q_in = source_term(dim,dim,L,L)
        A = build_matrix(dim,dim,L,L)
        if direct == true
            a = @benchmark solve_matrix($dim,$dim,$A,$Q_in)    
            n,logs = solve_matrix(dim,dim,A,Q_in)
            u = wrap(dim,dim,n)
        else
            a = @benchmark solve_matrix_cg($dim,$dim,$A,$Q_in,$tol)    
            n,logs = solve_matrix_cg(dim,dim,A,Q_in,tol)
            u = wrap(dim,dim,n)
        end
        norm = l2_norm(dim,dim,u,u_a)
        push!(times,mean(a.times))
        push!(norms,norm)
        push!(us,u)
    end
return times,norms,us
end;

## Solve the Poisson equation

In [ ]:
dims = [11,21,41,81,161,321]
L = 1
tol = 1e-8
times_cg,norms_cg,us_cg = benchmark_poisson(dims,L,tol,false);
times,norms,us = benchmark_poisson(dims,L,tol,true);

## Print the data to file

In [ ]:
file_names = ["times_cg","norms_cg","times","norms"]
files = [times_cg,norms_cg,us_cg,times,norms,us]
for i in range(1,length(file_names))
    open(file_names[i]*"_jl.txt", "w") do io
         writedlm(io, [files[i]],',')
    end
end


i=0
for u in us
    CSV.write("us_jl_"*string(i)*".csv", Tables.table(u), writeheader=false)
    i+=1
end